# Load packages

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import PIL
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
import scipy
import gc
from PIL import Image
# !pip install scikit-image
import pydicom
from sklearn.cluster import KMeans
from skimage import morphology
from skimage import measure
from skimage.transform import resize

PATH = "/Users/paritoshgupta/Desktop/aamlp/aamlp/input/osic-pulmonary-fibrosis-progression/"
OUTPUT_PATH = "/Users/paritoshgupta/Desktop/aamlp/aamlp/input/osic-pulmonary-fibrosis-progression/ads/"
print(f"# Changing directory to {PATH}")
# os.chdir(PATH)
print(f"\n** Files present in PATH --> {list(os.listdir(PATH))}")
PATH

# Changing directory to /Users/paritoshgupta/Desktop/aamlp/aamlp/input/osic-pulmonary-fibrosis-progression/

** Files present in PATH --> ['ads', '.DS_Store', 'archive', 'test', 'test.csv', 'train', 'train.csv', 'ads_4_folds.csv', 'ads.zip', 'sample_submission.csv']


'/Users/paritoshgupta/Desktop/aamlp/aamlp/input/osic-pulmonary-fibrosis-progression/'

# ADS Prepare

In [2]:
train = pd.read_csv(f"{PATH}/train.csv")
print(train.shape)
train_tmp1 = train.groupby(['Patient', 'Weeks']).mean().reset_index()
train_tmp2 = train[['Patient', 'Weeks', 'Sex', 'SmokingStatus']].drop_duplicates().reset_index(drop=True)
print(train_tmp1.shape)
print(train_tmp1.shape)
train = pd.merge(train_tmp1, train_tmp2, on=["Patient", "Weeks"], how="inner")
print(train.shape)
test = pd.read_csv(f"{PATH}/test.csv")

print("add infos")
sub = pd.read_csv(f"{PATH}/sample_submission.csv")
sub['Patient'] = sub['Patient_Week'].apply(lambda x:x.split('_')[0])
sub['Weeks'] = sub['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
sub =  sub[['Patient','Weeks','Confidence','Patient_Week']]
sub = sub.merge(test.drop('Weeks', axis=1), on="Patient")
train['WHERE'] = 'train'
test['WHERE'] = 'val'
sub['WHERE'] = 'test'
data = train.append([test, sub])
data['min_week'] = data['Weeks']
data.loc[data.WHERE=='test','min_week'] = np.nan
data['min_week'] = data.groupby('Patient')['min_week'].transform('min')
base = data.loc[data.Weeks == data.min_week]
base = base[['Patient','FVC']].copy()
base.columns = ['Patient','min_FVC']
base['nb'] = 1
base['nb'] = base.groupby('Patient')['nb'].transform('cumsum')
base = base[base.nb==1]
base.drop('nb', axis=1, inplace=True)
data = data.merge(base, on='Patient', how='left')
data['base_week'] = data['Weeks'] - data['min_week']
del base
COLS = ['Sex','SmokingStatus'] #,'Age'
FE = ['Patient']
for col in COLS:
    for mod in data[col].unique():
        FE.append(mod)
        data[mod] = (data[col] == mod).astype(int)
#=================
data['age'] = (data['Age'] - data['Age'].min() ) / ( data['Age'].max() - data['Age'].min() )
data['base_fvc'] = (data['min_FVC'] - data['min_FVC'].min() ) / ( data['min_FVC'].max() - data['min_FVC'].min() )
data['week'] = (data['base_week'] - data['base_week'].min() ) / ( data['base_week'].max() - data['base_week'].min() )
data['percent'] = (data['Percent'] - data['Percent'].min() ) / ( data['Percent'].max() - data['Percent'].min() )
FE += ['age', 'week','base_fvc', 'FVC']
print(data.columns)
data
# print(FE)
# data[FE]

(1549, 7)
(1542, 5)
(1542, 5)
(1542, 7)
add infos
Index(['Patient', 'Weeks', 'FVC', 'Percent', 'Age', 'Sex', 'SmokingStatus',
       'WHERE', 'Confidence', 'Patient_Week', 'min_week', 'min_FVC',
       'base_week', 'Male', 'Female', 'Ex-smoker', 'Never smoked',
       'Currently smokes', 'age', 'base_fvc', 'week', 'percent'],
      dtype='object')


,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,WHERE,Confidence,Patient_Week,...,base_week,Male,Female,Ex-smoker,Never smoked,Currently smokes,age,base_fvc,week,percent
0,ID00007637202177411956430,-4,2315.0,58.253649,79.0,Male,Ex-smoker,train,NaN,NaN,...,0.0,1,0,1,0,0,0.769231,0.241456,0.179012,0.236393
1,ID00007637202177411956430,5,2214.0,55.712129,79.0,Male,Ex-smoker,train,NaN,NaN,...,9.0,1,0,1,0,0,0.769231,0.241456,0.234568,0.215941
2,ID00007637202177411956430,7,2061.0,51.862104,79.0,Male,Ex-smoker,train,NaN,NaN,...,11.0,1,0,1,0,0,0.769231,0.241456,0.246914,0.184960
3,ID00007637202177411956430,9,2144.0,53.950679,79.0,Male,Ex-smoker,train,NaN,NaN,...,13.0,1,0,1,0,0,0.769231,0.241456,0.259259,0.201767
4,ID00007637202177411956430,11,2069.0,52.063412,79.0,Male,Ex-smoker,train,NaN,NaN,...,15.0,1,0,1,0,0,0.769231,0.241456,0.271605,0.186580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,ID00426637202313170790466,129,2925.0,71.824968,73.0,Male,Never smoked,test,100.0,ID00426637202313170790466_129,...,129.0,1,0,0,1,0,0.615385,0.354755,0.975309,0.345604
2273,ID00426637202313170790466,130,2925.0,71.824968,73.0,Male,Never smoked,test,100.0,ID00426637202313170790466_130,...,130.0,1,0,0,1,0,0.615385,0.354755,0.981481,0.345604
2274,ID00426637202313170790466,131,2925.0,71.824968,73.0,Male,Never smoked,test,100.0,ID00426637202313170790466_131,...,131.0,1,0,0,1,0,0.615385,0.354755,0.987654,0.345604
2275,ID00426637202313170790466,132,2925.0,71.824968,73.0,Male,Never smoked,test,100.0,ID00426637202313170790466_132,...,132.0,1,0,0,1,0,0.615385,0.354755,0.993827,0.345604


In [3]:
data[data.Patient=="ID00419637202311204720264"][['Patient', 'Weeks', 'FVC', 'Percent', 'Age', 'Sex', 'SmokingStatus',
       'WHERE', 'Confidence', 'Patient_Week', 'min_week', 'min_FVC',
       'base_week', 'age', 'base_fvc', 'week', 'percent']]



,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,WHERE,Confidence,Patient_Week,min_week,min_FVC,base_week,age,base_fvc,week,percent
1498,ID00419637202311204720264,6,3020.0,70.186855,73.0,Male,Ex-smoker,train,NaN,NaN,6.0,3020.0,0.0,0.615385,0.3724,0.179012,0.332421
1499,ID00419637202311204720264,7,2859.0,66.445106,73.0,Male,Ex-smoker,train,NaN,NaN,6.0,3020.0,1.0,0.615385,0.3724,0.185185,0.302311
1500,ID00419637202311204720264,9,2783.0,64.678814,73.0,Male,Ex-smoker,train,NaN,NaN,6.0,3020.0,3.0,0.615385,0.3724,0.197531,0.288097
1501,ID00419637202311204720264,10,2719.0,63.191410,73.0,Male,Ex-smoker,train,NaN,NaN,6.0,3020.0,4.0,0.615385,0.3724,0.203704,0.276128
1502,ID00419637202311204720264,13,2738.0,63.632983,73.0,Male,Ex-smoker,train,NaN,NaN,6.0,3020.0,7.0,0.615385,0.3724,0.222222,0.279682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1688,ID00419637202311204720264,129,3020.0,70.186855,73.0,Male,Ex-smoker,test,100.0,ID00419637202311204720264_129,6.0,3020.0,123.0,0.615385,0.3724,0.938272,0.332421
1689,ID00419637202311204720264,130,3020.0,70.186855,73.0,Male,Ex-smoker,test,100.0,ID00419637202311204720264_130,6.0,3020.0,124.0,0.615385,0.3724,0.944444,0.332421
1690,ID00419637202311204720264,131,3020.0,70.186855,73.0,Male,Ex-smoker,test,100.0,ID00419637202311204720264_131,6.0,3020.0,125.0,0.615385,0.3724,0.950617,0.332421
1691,ID00419637202311204720264,132,3020.0,70.186855,73.0,Male,Ex-smoker,test,100.0,ID00419637202311204720264_132,6.0,3020.0,126.0,0.615385,0.3724,0.956790,0.332421


In [4]:
data.shape

data[data.FVC == data.min_FVC].shape

(915, 22)

## meta data

In [5]:
df_patient = pd.read_csv(f"{OUTPUT_PATH}/osic_final_meta_data.csv")
print(df_patient.shape)
BAD_ID = ["ID00011637202177653955184", "ID00052637202186188008618"]
train_patients = list(df_patient[df_patient.label=="train"].Patient.unique())
print(len(train_patients))
train_patients = [x for x in train_patients if x not in BAD_ID]
print(len(train_patients))
test_patients = list(df_patient[df_patient.label=="test"].Patient.unique())
print(len(test_patients))

(176, 19)
171
169
5


## folds data

In [6]:
N_FOLDS = 5
df_folds = data.loc[data.WHERE=='train'][FE]
df_folds = df_folds[df_folds.Patient.isin(train_patients)].reset_index(drop=True)
print(df_folds.Patient.nunique())
print(df_folds.shape)
group_k_fold = GroupKFold(n_splits=N_FOLDS)
groups = df_folds['Patient'].values
for fold, (train_index, valid_index) in enumerate(group_k_fold.split(df_folds, df_folds.FVC, groups)):
    df_folds.loc[valid_index, 'fold'] = int(fold)
df_folds['fold'] = df_folds['fold'].astype(int)
df_folds.to_csv(f"{OUTPUT_PATH}/osic_5_folds.csv", index=False)

169
(1482, 10)


In [7]:
for fold in range(5):
    v_p = df_folds[df_folds.fold==fold].Patient.unique()
    t_p = df_folds[df_folds.fold!=fold].Patient.unique()
    common_patients = [x for x in t_p if x in v_p]
    print(len(common_patients))

0
0
0
0
0


In [8]:
valid_patients = df_folds[df_folds.fold==1].Patient.unique()
train_patients = df_folds[df_folds.fold!=1].Patient.unique()
valid_patients
# df_folds[df_folds.fold==0].Patient

array(['ID00007637202177411956430', 'ID00010637202177584971671',
       'ID00030637202181211009029', 'ID00035637202182204917484',
       'ID00048637202185016727717', 'ID00060637202187965290703',
       'ID00078637202199415319443', 'ID00093637202205278167493',
       'ID00102637202206574119190', 'ID00115637202211874187958',
       'ID00122637202216437668965', 'ID00125637202218590429387',
       'ID00128637202219474716089', 'ID00139637202231703564336',
       'ID00165637202237320314458', 'ID00180637202240177410333',
       'ID00196637202246668775836', 'ID00213637202257692916109',
       'ID00216637202257988213445', 'ID00229637202260254240583',
       'ID00233637202260580149633', 'ID00240637202264138860065',
       'ID00242637202264759739921', 'ID00249637202266730854017',
       'ID00298637202280361773446', 'ID00312637202282607344793',
       'ID00317637202283194142136', 'ID00331637202286306023714',
       'ID00341637202287410878488', 'ID00364637202296074419422',
       'ID003766372022976

## test data

In [9]:
df_test = data.loc[data.WHERE=='train'][FE]
df_test = df_test[df_test.Patient.isin(test_patients)].reset_index(drop=True)
print(df_test.Patient.nunique())
print(df_test.shape)
df_test.to_csv(f"{OUTPUT_PATH}/osic_test_junk.csv", index=False)


5
(44, 10)


## submission data

In [45]:
df_sub = data.loc[data.WHERE=='test'][FE]
df_sub_mapping = data.loc[data.WHERE=='test'][['Patient', 'Weeks', 'week']]
df_sub = df_sub[df_sub.Patient.isin(test_patients)].reset_index(drop=True)
print(df_sub.Patient.nunique())
print(df_sub.shape)
print(df_sub_mapping.shape)
df_sub.to_csv(f"{OUTPUT_PATH}/osic_submission.csv", index=False)
df_sub_mapping.to_csv(f"{OUTPUT_PATH}/osic_submission_mapping.csv", index=False)


5
(730, 10)
(730, 3)


In [13]:
data.loc[data.WHERE=='test']


,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,WHERE,Confidence,Patient_Week,...,base_week,Male,Female,Ex-smoker,Never smoked,Currently smokes,age,base_fvc,week,percent
1547,ID00419637202311204720264,-12,3020.0,70.186855,73.0,Male,Ex-smoker,test,100.0,ID00419637202311204720264_-12,...,-18.0,1,0,1,0,0,0.615385,0.372400,0.067901,0.332421
1548,ID00419637202311204720264,-11,3020.0,70.186855,73.0,Male,Ex-smoker,test,100.0,ID00419637202311204720264_-11,...,-17.0,1,0,1,0,0,0.615385,0.372400,0.074074,0.332421
1549,ID00419637202311204720264,-10,3020.0,70.186855,73.0,Male,Ex-smoker,test,100.0,ID00419637202311204720264_-10,...,-16.0,1,0,1,0,0,0.615385,0.372400,0.080247,0.332421
1550,ID00419637202311204720264,-9,3020.0,70.186855,73.0,Male,Ex-smoker,test,100.0,ID00419637202311204720264_-9,...,-15.0,1,0,1,0,0,0.615385,0.372400,0.086420,0.332421
1551,ID00419637202311204720264,-8,3020.0,70.186855,73.0,Male,Ex-smoker,test,100.0,ID00419637202311204720264_-8,...,-14.0,1,0,1,0,0,0.615385,0.372400,0.092593,0.332421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,ID00426637202313170790466,129,2925.0,71.824968,73.0,Male,Never smoked,test,100.0,ID00426637202313170790466_129,...,129.0,1,0,0,1,0,0.615385,0.354755,0.975309,0.345604
2273,ID00426637202313170790466,130,2925.0,71.824968,73.0,Male,Never smoked,test,100.0,ID00426637202313170790466_130,...,130.0,1,0,0,1,0,0.615385,0.354755,0.981481,0.345604
2274,ID00426637202313170790466,131,2925.0,71.824968,73.0,Male,Never smoked,test,100.0,ID00426637202313170790466_131,...,131.0,1,0,0,1,0,0.615385,0.354755,0.987654,0.345604
2275,ID00426637202313170790466,132,2925.0,71.824968,73.0,Male,Never smoked,test,100.0,ID00426637202313170790466_132,...,132.0,1,0,0,1,0,0.615385,0.354755,0.993827,0.345604


In [9]:
df_folds.columns

Index(['Patient', 'Male', 'Female', 'Ex-smoker', 'Never smoked',
       'Currently smokes', 'age', 'week', 'base_fvc', 'FVC', 'fold'],
      dtype='object')

In [15]:
df_folds.loc[[0,1,2,4], 'Patient'].values

array(['ID00007637202177411956430', 'ID00007637202177411956430',
       'ID00007637202177411956430', 'ID00007637202177411956430'],
      dtype=object)

In [35]:
df_folds.shape[1]

11

In [37]:
fold = 0
df_folds = pd.read_csv(f"{OUTPUT_PATH}/osic_10_folds.csv")
df_train = df_folds[df_folds.fold !=fold].reset_index(drop=True)
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train.shape
int(np.floor(df_train.shape[0])/32)

41

In [38]:
int(n.floor(df_train.shape[0]/32))

41

In [40]:
df_train.shape[0]/32

41.84375